#Garvika 202318061
#Aditi 202318005
#Jyoti 202318049
#Kunj 202103018

#1. Installation

In [1]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163004 sha256=0498b21b03deb5f5b346ba2d341d5839612e9b12e330c1ce1fc0a5f5f90cf1ce
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import numpy as np
import pandas as pd
import re
from scipy.sparse import csr_matrix
import random
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise import SVD
from surprise.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#2. Data Preprocessing

In [276]:
# Load Myntra dataset
myntra = pd.read_csv('/content/drive/MyDrive/Collaborative/myntra_excel.csv',usecols=[2,9])
myntra.head(1)

,product_id,title
0,6937673,IMARA Women Black Solid Top


In [277]:
myntra

,product_id,title
0,6937673,IMARA Women Black Solid Top
1,7441182,House of Pataudi Men Black Printed Straight Kurta
2,9245141,The White Willow Unisex Off-White Therapedic M...
3,1314889,Dupatta Bazaar Pink Dupatta
4,7705322,Manyavar Men Yellow & White Self Design Kurta ...
...,...,...
3438,8317947,Shree Women Black & Pink Printed Tunic
3439,8480717,DEYANN Men Copper-Toned & Black Printed Kurta ...
3440,4703072,Shree Grey & Black Checked Tunic
3441,9744027,Ishin Women Off-White & Pink Embroidered Flare...


In [278]:
# Removing duplicate entries based on title from myntra
myntra  = myntra.drop_duplicates(subset=['title'])
myntra.shape


(3006, 2)

In [279]:
# After removing duplicate values, we reset index
myntra.reset_index(drop=True, inplace = True)

In [280]:
# All the features of the dataset
features = myntra.columns
print(features)

Index(['product_id', 'title'], dtype='object')


In [281]:
# To check if the dataset has any null values or not
myntra.isnull().sum()

product_id    0
title         0
dtype: int64

In [282]:
# First row
myntra.loc[0]

product_id                        6937673
title         IMARA Women Black Solid Top
Name: 0, dtype: object

In [283]:
x=myntra['product_id']
x[3000:3200]

3000    4369937
3001    7684345
3002    9254221
3003    8317947
3004    9744027
3005    2106414
Name: product_id, dtype: int64

In [284]:
# Dropping duplicate products based on product_id
myntra = myntra.drop_duplicates(subset = ["product_id"])
myntra.shape

(3006, 2)

In [291]:
#Checking duplicates
myntra['product_id'].value_counts()

0       1
2007    1
1998    1
1999    1
2000    1
       ..
1004    1
1005    1
1006    1
1007    1
3005    1
Name: product_id, Length: 3006, dtype: int64

In [292]:
#index reset
myntra.reset_index(drop = True, inplace = True)

In [287]:
# Removing all null values
myntra.dropna(inplace = True)

In [288]:
myntra.reset_index(inplace = True)

In [289]:
# Product_id == index
myntra['product_id'] = myntra['index']


In [290]:
myntra = myntra.drop_duplicates(subset=['title'])
myntra.shape

(3006, 3)

In [296]:
# Ratings dataset
ratings = pd.read_csv('/content/drive/MyDrive/Collaborative/ratings.csv')

In [297]:
# Dropping unnecessery column
ratings.drop('Unnamed: 0' , axis =1 , inplace = True)

In [298]:
# merging ratings dataset with myntra using product_id
merged_data = ratings.merge(myntra , on="product_id")
merged_data.head(5)

,userId,product_id,Rating,index,title
0,3269,0,4.30,0,IMARA Women Black Solid Top
1,4575,0,3.95,0,IMARA Women Black Solid Top
2,3844,0,3.72,0,IMARA Women Black Solid Top
3,1260,0,3.98,0,IMARA Women Black Solid Top
4,5298,0,4.37,0,IMARA Women Black Solid Top


In [299]:
merged_data=merged_data.dropna(axis=0,subset=['title'])
rating_count = merged_data.groupby('title')['Rating'].count().reset_index().rename(columns={'Rating': 'totalratings'})[['title', 'totalratings']]
merged_data=merged_data.merge(rating_count, on='title')

In [300]:
merged_data.describe()

,userId,product_id,Rating,index,totalratings
count,386828.000000,386828.000000,386827.000000,386828.000000,386828.000000
mean,4015.218849,700.579319,4.250319,700.579319,276.966784
std,2307.646286,404.974278,0.433396,404.974278,16.612261
min,1.000000,0.000000,3.500000,0.000000,96.000000
25%,2016.000000,349.000000,3.870000,349.000000,266.000000
50%,4043.000000,701.000000,4.250000,701.000000,277.000000
75%,6008.000000,1052.000000,4.630000,1052.000000,288.000000
max,7999.000000,1401.000000,5.000000,1401.000000,341.000000


###We can see that the mean value of number of total ratings assigned to a given product 275. So items that are being rated more than 275 can be taken into consideration.

###Devising a Popularity Threshold

In [301]:
popular_threshold=275
rating_threshold=4
popular_items = merged_data.query('totalratings >= @popular_threshold')


In [302]:
popular_items.drop(['index'], axis=1)

,userId,product_id,Rating,title,totalratings
492,6111,2,3.57,The White Willow Unisex Off-White Therapedic M...,287
493,4954,2,3.79,The White Willow Unisex Off-White Therapedic M...,287
494,4749,2,4.93,The White Willow Unisex Off-White Therapedic M...,287
495,4953,2,4.77,The White Willow Unisex Off-White Therapedic M...,287
496,6653,2,3.55,The White Willow Unisex Off-White Therapedic M...,287
...,...,...,...,...,...
386454,579,1399,3.89,Anouk Women Navy Blue & Off-White Printed A-Li...,296
386455,304,1399,4.03,Anouk Women Navy Blue & Off-White Printed A-Li...,296
386456,1454,1399,4.10,Anouk Women Navy Blue & Off-White Printed A-Li...,296
386457,1943,1399,3.55,Anouk Women Navy Blue & Off-White Printed A-Li...,296


In [303]:
popular_items=popular_items.drop_duplicates(subset=['userId','title','Rating','totalratings'])

In [305]:
popular_items.head(5)

,userId,product_id,Rating,index,title,totalratings
492,6111,2,3.57,2,The White Willow Unisex Off-White Therapedic M...,287
493,4954,2,3.79,2,The White Willow Unisex Off-White Therapedic M...,287
494,4749,2,4.93,2,The White Willow Unisex Off-White Therapedic M...,287
495,4953,2,4.77,2,The White Willow Unisex Off-White Therapedic M...,287
496,6653,2,3.55,2,The White Willow Unisex Off-White Therapedic M...,287


#3. Memory Based collaborative filtering

##1. USER_USER Based

In [306]:
df = popular_items.groupby(['userId', 'product_id']).mean().reset_index()
df

<ipython-input-306-b8d988cb3a67>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = popular_items.groupby(['userId', 'product_id']).mean().reset_index()


,userId,product_id,Rating,index,totalratings
0,2,4,4.30,4.0,291.0
1,2,26,4.05,26.0,301.0
2,2,68,4.12,68.0,277.0
3,2,84,3.54,84.0,289.0
4,2,88,4.32,88.0,292.0
...,...,...,...,...,...
213234,7999,1216,4.70,1216.0,293.0
213235,7999,1258,4.59,1258.0,295.0
213236,7999,1271,3.51,1271.0,286.0
213237,7999,1289,3.93,1289.0,290.0


###Train-Test-Split

In [307]:
from sklearn.model_selection import train_test_split

#Assigning X as the original ratings dataframe and y as the user_id column of ratings.
X = df.copy()
y = df['userId']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state=42)

###Evaluation Metric -RMSE

In [308]:
from sklearn.metrics import mean_squared_error

#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def baseline(user_id, movie_id):
    return 3.0

In [309]:
def score(cf_model):
    # Constructing a list of user-movie tuples from the testing dataset
    id_pairs = zip(X_test['userId'], X_test['product_id'])

    # Predicting the rating for every user-movie data point
    y_pred = np.array([cf_model(user, item) for (user, item) in id_pairs])

    # Actual ratings given by the users in the test data
    y_true = np.array(X_test['Rating'])

    user_item_df = pd.DataFrame({
        'userId': X_test['userId'],
        'product_id': X_test['product_id'],
        'Rating': y_true,
        'Pred_Ratings': y_pred
    })

    # Final RMSE score and user-item DataFrame
    return rmse(y_true, y_pred), user_item_df


###User-User pivot **creation**

In [310]:
import pandas as pd
train_matrix = df.pivot(index='userId', columns='product_id', values='Rating')
train_matrix.head()
#Creating a dummy ratings matrix with all null values imputed to 0
train_matrix_dummy = train_matrix.copy().fillna(0)

###Using Cosaine Similarity to generate a similarity weights

In [311]:
from sklearn.metrics.pairwise import cosine_similarity
#Computing cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(train_matrix_dummy, train_matrix_dummy)
cosine_sim = pd.DataFrame(cosine_sim, index=train_matrix.index, columns=train_matrix.index)
cosine_sim.head(10)

userId,2,3,4,5,6,7,9,10,11,12,...,7990,7991,7992,7993,7994,7995,7996,7997,7998,7999
userId,,,,,,,,,,,,,,,,,,,,,
2,1.000000,0.000000,0.046011,0.054331,0.070888,0.033131,0.0,0.031536,0.025154,0.034182,...,0.027476,0.000000,0.023522,0.101581,0.091138,0.021530,0.092650,0.000000,0.095994,0.023566
3,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.230569,...,0.116943,0.000000,0.000000,0.000000,0.000000,0.100825,0.000000,0.000000,0.000000,0.000000
4,0.046011,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.086770,0.000000,0.047857,0.000000,0.000000,0.155718,0.000000,0.000000
5,0.054331,0.000000,0.000000,1.000000,0.000000,0.087625,0.0,0.113464,0.000000,0.115936,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.134871,0.000000,0.000000,0.000000,0.000000
6,0.070888,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.084912,0.000000,0.000000,0.000000,0.103175,0.000000
7,0.033131,0.000000,0.000000,0.087625,0.000000,1.000000,0.0,0.032522,0.000000,0.000000,...,0.072997,0.000000,0.086451,0.000000,0.035102,0.034114,0.000000,0.000000,0.081951,0.023467
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.065037,0.000000,0.042658,0.100785,0.000000,0.000000,0.051519
10,0.031536,0.000000,0.000000,0.113464,0.000000,0.032522,0.0,1.000000,0.000000,0.000000,...,0.052862,0.000000,0.053444,0.078805,0.000000,0.063575,0.000000,0.059551,0.051086,0.026207
11,0.025154,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,...,0.082482,0.086776,0.027695,0.118699,0.034996,0.057246,0.000000,0.000000,0.034862,0.053587


###Collaborative filtering model

In [312]:
#User Based Collaborative Filter using Cosine Similarity
def cf_user_wmean(user_id, product_id):
    if product_id in train_matrix:
        sim_scores = cosine_sim[user_id]#Similarity scores for the user in question with every other user
        m_ratings = train_matrix[product_id] #User ratings for the movie in question
        idx = m_ratings[m_ratings.isnull()].index #Extracting the indices containing NaN in the m_ratings series
        m_ratings = m_ratings.dropna()#Extracting the indices containing NaN in the m_ratings series
        sim_scores = sim_scores.drop(idx)
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()#Final weighted mean

    else:
        wmean_rating = 3.0 #Defaulting to a rating of 3.0 in the absence of any information
    return wmean_rating

In [313]:
user_id_random= np.random.choice(df['userId'])
prod_id_random= np.random.choice(df['product_id'])

###Generating final RMSE score and Predicted Ratings

In [314]:
rmse_score,user_item_df = score(cf_user_wmean)

In [315]:
print("RMSE SCORE::")
rmse_score

RMSE SCORE::


0.41067806227913844

In [316]:
print('Predicted Ratings DataFrame::')
user_item_df

Predicted Ratings DataFrame::


,userId,product_id,Rating,Pred_Ratings
29875,1130,1263,4.19,4.315221
56258,2121,173,4.70,4.263475
171063,6426,690,3.57,4.256582
80295,3039,623,4.69,4.289595
79391,3000,612,4.45,4.269291
...,...,...,...,...
94686,3570,1140,3.72,4.229812
111050,4194,844,3.90,4.220760
72025,2719,1364,4.31,4.235851
176154,6617,15,4.26,4.246974


###Generating top 10 recommendations

In [317]:
def get_top_n_neighbors(user_id, similarity_matrix, n=10):
    user_similarity_scores = similarity_matrix.loc[user_id] # Getting similarity scores for the given user
    top_n_neighbors = user_similarity_scores.sort_values(ascending=False).head(n) # Getting the top N similar users
    return top_n_neighbors.index

def get_top_n_neighbor_recommendations(user_id, cf_model, similarity_matrix, n=10):
    top_n_neighbors = get_top_n_neighbors(user_id, similarity_matrix, n=n)    # Getting the top N neighbors for the user
    all_product_ids = train_matrix.columns # Getting all product IDs in the dataset
    aggregate_ratings = {} #to stoore predicted ratings

    for neighbor_id in top_n_neighbors:
        neighbor_ratings = [cf_model(neighbor_id, product_id) for product_id in all_product_ids] # Predicted ratings for all products for the neighbor
        for product_id, rating in zip(all_product_ids, neighbor_ratings):  # Aggregate of the ratings for each product
            if product_id not in aggregate_ratings:
                aggregate_ratings[product_id] = []
            aggregate_ratings[product_id].append(rating)

    avg_ratings = {product_id: np.mean(ratings) for product_id, ratings in aggregate_ratings.items()} # Average predicted rating for each product

    sorted_products = sorted(avg_ratings.items(), key=lambda x: x[1], reverse=True)

    # Top N recommended products
    top_n_recommendations = pd.DataFrame(sorted_products[:n], columns=['product_id', 'predicted_rating'])

    return top_n_recommendations


In [318]:
#Generating a random user id to reccomend
user_id_to_recommend = np.random.choice(X_test['userId'])
user_id_to_recommend

100

###Getting recommended items DataFrame

In [319]:
top_neighbor_recommendations = get_top_n_neighbor_recommendations(user_id_to_recommend, cf_user_wmean, cosine_sim, n=10)

prod_ids = top_neighbor_recommendations['product_id']
titles_of_prod_ids = [popular_items.loc[popular_items['product_id'] == i, 'title'].iloc[0] for i in prod_ids]
top_neighbor_recommendations['title'] = titles_of_prod_ids

top_neighbor_recommendations


,product_id,predicted_rating,title
0,1311,4.340648,Aj DEZInES Boys Turquoise Blue & Black Solid S...
1,419,4.339207,RAJUBHAI HARGOVINDAS Men Beige Solid Straight ...
2,1068,4.338409,BownBee Girls Pink & Yellow Printed Kurta with...
3,335,4.335503,Shree Women White Solid Wide Leg Schiffli Embr...
4,1019,4.332064,United Colors of Benetton Boys Navy Nehru Jacket
5,92,4.331758,HERE&NOW Women Teal Green Printed Flared Palazzos
6,515,4.325704,Abhiyuthan Men Blue & White Striped Straight K...
7,139,4.324688,VASTRAMAY Men Navy Blue Self Design Kurta with...
8,603,4.324623,BIBA Girls Orange & Brown Printed Salwar Suit ...
9,163,4.323484,Anouk Women Olive Green & Golden Printed A-Lin...


##2. ITEM_ITEM based

In [321]:
from sklearn.model_selection import train_test_split

#Assigning X as the original ratings dataframe and y as the user_id column of ratings.
X = df.copy()
y = df['product_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state=42)

In [322]:
# Transpose of the train_matrix_dummy so that items are on rows and users on columns
item_similarity_matrix = cosine_similarity(train_matrix_dummy.T, train_matrix_dummy.T)
item_cosine_sim = pd.DataFrame(item_similarity_matrix, index=train_matrix.columns, columns=train_matrix.columns)
item_cosine_sim.head(10)


product_id,2,4,6,9,10,13,15,16,17,19,...,1388,1390,1392,1393,1394,1395,1396,1397,1398,1399
product_id,,,,,,,,,,,,,,,,,,,,,
2,1.000000,0.040027,0.053477,0.032842,0.050206,0.033705,0.057687,0.051792,0.046127,0.039681,...,0.041564,0.052237,0.025497,0.066701,0.059737,0.050861,0.048234,0.066242,0.025106,0.052643
4,0.040027,1.000000,0.050926,0.050192,0.069837,0.059998,0.044331,0.032808,0.041576,0.028857,...,0.029782,0.051781,0.049495,0.036774,0.064723,0.028803,0.039793,0.025554,0.044620,0.019427
6,0.053477,0.050926,1.000000,0.038722,0.038865,0.050687,0.030593,0.046874,0.036587,0.036024,...,0.046396,0.037703,0.033114,0.026035,0.045488,0.025875,0.056269,0.060450,0.060157,0.027604
9,0.032842,0.050192,0.038722,1.000000,0.047491,0.043555,0.045025,0.058858,0.058738,0.052680,...,0.040599,0.074087,0.050922,0.044530,0.060964,0.065596,0.068219,0.030844,0.055219,0.053918
10,0.050206,0.069837,0.038865,0.047491,1.000000,0.029515,0.059106,0.045356,0.060841,0.045296,...,0.046433,0.049034,0.070670,0.043327,0.057068,0.022777,0.040069,0.052276,0.035064,0.055513
13,0.033705,0.059998,0.050687,0.043555,0.029515,1.000000,0.048246,0.033539,0.074882,0.051355,...,0.059056,0.043438,0.042138,0.038672,0.050743,0.054481,0.039482,0.021398,0.049304,0.039029
15,0.057687,0.044331,0.030593,0.045025,0.059106,0.048246,1.000000,0.052703,0.061271,0.035946,...,0.064352,0.045269,0.039539,0.033123,0.050498,0.059138,0.069795,0.055806,0.050078,0.068336
16,0.051792,0.032808,0.046874,0.058858,0.045356,0.033539,0.052703,1.000000,0.036239,0.062649,...,0.037821,0.066861,0.036008,0.030705,0.032618,0.068278,0.077907,0.050355,0.044055,0.045620
17,0.046127,0.041576,0.036587,0.058738,0.060841,0.074882,0.061271,0.036239,1.000000,0.044580,...,0.049390,0.043339,0.037321,0.052556,0.037596,0.024106,0.057577,0.069555,0.053441,0.049973


##Item based collaborative model

In [323]:
def cf_item_wmean(user_id, product_id):
    if product_id in train_matrix:
        sim_scores = item_cosine_sim[product_id]  # Similarity scores for the item in question with every other item
        u_ratings = train_matrix.loc[user_id]  # User ratings for the user in question
        idx = u_ratings[u_ratings.isnull()].index  # Extracting the indices containing NaN in the u_ratings series
        u_ratings = u_ratings.dropna()  # Dropping NaN values
        sim_scores = sim_scores.drop(idx)
        wmean_rating = np.dot(sim_scores, u_ratings) / sim_scores.sum()  # Final weighted mean

    else:
        wmean_rating = 3.0  # Default to a rating of 3.0 in the absence of any information
    return wmean_rating


###Evaluation -RMSE

In [324]:
rmse_score,user_item_df=score(cf_item_wmean)

In [325]:
rmse_score

0.26854352564561923

###Predicted ratings of the Items - Data Frame

In [330]:
print("Predicted ratings dataframe")
user_item_df.head()

Predicted ratings dataframe


,userId,product_id,Rating,Pred_Ratings
29875,1130,1263,4.19,4.191884
56258,2121,173,4.70,4.423555
171063,6426,690,3.57,4.059821
80295,3039,623,4.69,4.415343
79391,3000,612,4.45,4.412761


###Generating top 10 recommendations

In [331]:
#Generating a random user id to reccomend
product_id_to_recommend = np.random.choice(X_test['product_id'])
product_id_to_recommend

993

In [333]:
top_neighbor_recommendations = get_top_n_neighbor_recommendations(product_id_to_recommend, cf_item_wmean, cosine_sim, n=10)
prod_ids = top_neighbor_recommendations['product_id']
titles_of_prod_ids = [popular_items.loc[popular_items['product_id'] == i, 'title'].iloc[0] for i in prod_ids]
top_neighbor_recommendations['title'] = titles_of_prod_ids
print("Top 10 recommendations")
top_neighbor_recommendations

Top 10 recommendations


,product_id,predicted_rating,title
0,1244,4.390270,Prayyan Men Off-White Striped A-Line Kurta
1,362,4.336002,Bitiya by Bhama Girls Mustard Yellow & Pink Em...
2,1231,4.333595,KISAH Men Blue Solid Straight Kurta
3,256,4.313035,Anouk Women Black Woven Design Straight Kurta
4,1064,4.312654,IVOC Men Navy Blue Solid Straight Kurta
5,46,4.311787,Libas Women Grey & Orange Checked Kurta with P...
6,1236,4.310821,Shree Women Burgundy Solid Wide Leg Palazzos
7,1145,4.310634,Kids Round Neck Solid Color Kurta Embroidery O...
8,581,4.307968,Manish Creations Men Cream-Coloured Striped Ku...
9,436,4.307138,IMARA Fusion Women Off-White A-Line Kurta


# 4.Model Based Aproach

## 1. KNN

###Getting the data ready




In [176]:
import pandas as pd
from surprise import Dataset, Reader
columns = ['userId', 'title', 'Rating']
df = popular_items
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[columns], reader)


###Parameter Tuning

In [334]:

# Choosing the parameters for grid
param_grid = {'k': [8,10], 'min_k': [2,3], 'sim_options': {'name': ['cosine'], 'user_based': [True], 'min_support': [2]}}

#Knn basic object
knn_basic = KNNBasic()

#GridSearchCV object
grid_search = GridSearchCV(KNNBasic, param_grid, measures=['RMSE'], cv=3)
grid_search.fit(data)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [335]:
# Getting the best parameters
best_params = grid_search.best_params['rmse']
print("Best Parameters:", best_params)


Best Parameters: {'k': 10, 'min_k': 3, 'sim_options': {'name': 'cosine', 'user_based': True, 'min_support': 2}}


###Model Fitting

In [336]:
# Using the best parameters obtained from grid search
best_k = best_params['k']
best_min_k = best_params['min_k']
best_sim_options = best_params['sim_options']

# Implementing KNNBasic algorithm with the tuned parameters
best_knn_basic = KNNBasic(k=best_k, min_k=best_min_k, sim_options=best_sim_options)

trainset = data.build_full_trainset()

# Model Training
best_knn_basic.fit(trainset)

print("Model trained successfully!")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Model trained successfully!


###Top 10 Recommendations




In [355]:
#Generating a random user id to recommend
user_id_to_recommend = '7887'
all_items = list(trainset.all_items())

# Predictions for all items
predictions = [algo.predict(str(user_id_to_recommend), str(item)) for item in all_items]


In [344]:
df = popular_items.groupby(['userId', 'product_id']).mean().reset_index()

<ipython-input-344-e26cfdf83e48>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = popular_items.groupby(['userId', 'product_id']).mean().reset_index()


###Model Evaluation

In [233]:
# Evaluating the model
# Cross-validation
cv_results = cross_validate(best_knn_basic, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print("\nCross-validation results:")
for measure, values in cv_results.items():
    print(f"{measure}: {sum(values) / len(values)}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4542  0.4569  0.4569  0.4580  0.4554  0.4563  0.0013  
MAE (testset)     0.3873  0.3895  0.3900  0.3909  0.3884  0.3892  0.0013  
Fit time          12.21   11.20   9.38    9.03    8.63    10.09   1.38    
Test time         16.71   12.44   12.12   11.79   12.06   13.02   1.86    

Cross-validation results:
test_rmse: 0.4562656918619433
test_mae: 0.38921814498017515
fit_time: 10.089125442504884
test_time: 13.024549341201782


#2. Matrix Factorization using SVD

###Hyper Parameter Tuning



In [225]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import GridSearchCV

# Choosing the parameter grid for GridSearchCV
param_grid = {
    "n_epochs": [10, 20],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.02]
}

# Hyperparameter tuning
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=5)
gs.fit(data)

# Getting the best hyperparameters
best_params = gs.best_params["rmse"]




In [227]:
# Training the model with the tuned hyperparameters
algo = SVD(n_epochs=best_params['n_epochs'], lr_all=best_params['lr_all'], reg_all=best_params['reg_all'])
algo.fit(trainset)

###Running the Model

In [258]:
trainset = data.build_full_trainset()
algo = SVD()

###Making Recommendations

In [228]:
# Making predictions for a specific user
user_id_to_recommend = "610"
items_to_recommend = popular_items['title'].unique()

# Generating predictions for all items
predictions = [(item, algo.predict(user_id_to_recommend, item).est) for item in items_to_recommend]

In [235]:
top_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:10] # Sorting predictions by estimated rating (descending order)
#Fetching titles
top_recommendations_with_titles = [(item_id, popular_items[popular_items['title'] == item_id]['title'].values[0]) for item_id, _ in top_predictions]
title=[]
pred_rating=[]
# Top 10 recommendations with titles
print("Top 10 Recommendations:")

for x,y in top_predictions:
  title.append(x)
  pred_rating.append(y)

pred_df=pd.DataFrame({'title':title,'Pred_ratings':pred_rating})
pred_df

Top 10 Recommendations:


,title,Pred_ratings
0,Biba Women Orange Woven-Design Straight Kurta,4.361376
1,URBAN DREAM White Geometric Flat 210 TC Cotton...,4.354982
2,Global Desi Women Yellow & Olive Green Woven D...,4.336028
3,Biba Women Peach-Coloured Printed Straight Kurta,4.330253
4,RAJUBHAI HARGOVINDAS Men Green Woven Design St...,4.329880
5,PINKSKY Women Turquoise Blue & Pink Floral Pri...,4.327137
6,La Firangi Women Green & Grey Printed Straight...,4.325497
7,Libas Women Yellow Solid Straight Kurta,4.324950
8,YK Girls Pink Solid A-Line Kurta,4.320645
9,Global Desi Women Black Palazzos,4.319887


###Evaluation - Cross Validation

In [260]:
# Perform cross-validation on the entire dataset
cv_results = cross_validate(algo, data, cv=5, verbose=True)

# Display the cross-validation results
print("Cross-Validation Results on Entire Dataset:")
for key, value in cv_results.items():
    print(f"{key}: {value}")


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4479  0.4471  0.4469  0.4473  0.4472  0.4473  0.0003  
MAE (testset)     0.3843  0.3836  0.3829  0.3839  0.3837  0.3837  0.0005  
Fit time          3.78    4.00    3.63    4.58    3.92    3.98    0.33    
Test time         0.45    0.28    0.30    0.50    0.30    0.37    0.09    
Cross-Validation Results on Entire Dataset:
test_rmse: [0.44790249 0.44713923 0.44691619 0.44732718 0.44717386]
test_mae: [0.38431032 0.38364043 0.38287316 0.38387968 0.38365732]
fit_time: (3.777618885040283, 3.9999334812164307, 3.6265206336975098, 4.584957122802734, 3.915456533432007)
test_time: (0.4510016441345215, 0.28238368034362793, 0.3041839599609375, 0.49831366539001465, 0.2980082035064697)


############################# END ##############################################


